<a href="https://colab.research.google.com/github/francesco-mollica/word2vec-entity_embeddings/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"

# CHOI_path = base_dir + "corpus_data_CHOI.txt"
# FIGER_path = base_dir + "corpus_data_FIGER.txt"
# ontonotes_path = base_dir + "corpus_data_ontonotes.txt"
balanced_ontonotes_path = base_dir + "corpus_data_balanced_ontonotes.txt"
# bbn_path = base_dir + "corpus_data_bbn.txt"

Mounted at /content/drive/


In [68]:
import pickle
with open(balanced_ontonotes_path, "rb") as fp:
  b = pickle.load(fp)

In [69]:
from gensim.models import Word2Vec

In [87]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
            for line in open(base_dir + self.filename):
                yield line.split()
 
 # a memory-friendly iterator

In [88]:
sentences = MySentences("prova.txt")
w2v_model = Word2Vec(iter=1, min_count=1)

In [89]:
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(232, 1290)

In [90]:
w2v_model.wv.most_similar(positive=["/PERSON"])

[('.', 0.19030408561229706),
 ('only', 0.17871451377868652),
 ('way', 0.1505899429321289),
 ('could', 0.10395622998476028),
 ('them', 0.10206874459981918),
 ('was', 0.09195896983146667),
 ('/PERSON/TITLE', 0.09068597108125687),
 ('you', 0.07853254675865173),
 ('get', 0.07625815272331238),
 ('and', 0.07206336408853531)]